# Supporting material for "Glaberish: Generalizing the Continuously-Valued Lenia Framework to Arbitrary

In [ ]:
# this section needs to be run for setup on colab
# uncomment and run the commands below if on colab
# then restart the runtime and run tests in the next cell


#! git clone https://github.com/riveSunder/yuca.git my_yuca
#%cd my_yuca
#! pip install -e .

In [ ]:
#! python -m testing.test_all

In [ ]:
import os
import copy
import numpy as np
import scipy
import torch

import matplotlib
import matplotlib.animation 
import matplotlib.pyplot as plt

my_cmap = plt.get_cmap("magma")
plt.rcParams['figure.dpi'] = 300

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
matplotlib.rcParams["animation.embed_limit"] = 256
matplotlib.rcParams["font.size"] = 10
#matplotlib.rcParams["text.usetex"] = True

import IPython

import skimage
import skimage.io as sio
import skimage.transform

import notebooks.my_helpers as my_helpers
from importlib import reload
reload(my_helpers)

import yuca
from yuca.utils import seed_all
from yuca.patterns import get_puffer
from yuca.multiverse import CA
from yuca.lenia import Lenia
from yuca.zoo.librarian import Librarian


from yuca.analyze import get_spatial_entropy, \
        get_multiconditional_grid_entropy, \
        get_conditional_grid_entropy,\
        get_grid_entropy

def get_center_of_mass(grid, spacing=20):
    
    if len(grid.shape) > 2:
        my_grid = np.array(grid.cpu().squeeze())
    else:
        my_grid = np.array(1.0 * grid.cpu())
        
    edge_l = my_grid[:,:spacing].sum()
    edge_r = my_grid[:,-spacing:].sum()
    edge_t = my_grid[:spacing,:].sum()
    edge_b = my_grid[-spacing:,:].sum()
    
    max_edge = np.max([edge_l, edge_r, edge_t, edge_b])
    
    xx_grid, yy_grid = np.meshgrid(np.arange(my_grid.shape[-1]), np.arange(my_grid.shape[-2]))
    
    if max_edge:
        
        if edge_r == max_edge:
            com_x = xx_grid[:, spacing:] * my_grid[:, spacing:] / my_grid[:, spacing:].sum()
            com_y = yy_grid[:, spacing:] * my_grid[:, spacing:] / my_grid[:, spacing:].sum()
            
        elif edge_l == max_edge:
            com_x = xx_grid[:, :-spacing] * my_grid[:, :-spacing] /  my_grid[:, :-spacing].sum()
            com_y = yy_grid[:, :-spacing] * my_grid[:, :-spacing] /  my_grid[:, :-spacing].sum()
            
        elif edge_t == max_edge:
            com_x = xx_grid[:-spacing, :] * my_grid[:-spacing, :] / my_grid[:-spacing, :].sum()
            com_y = yy_grid[:-spacing, :] * my_grid[:-spacing, :] /my_grid[:-spacing, :].sum()
            
        else: # edge_b == max_edge:
            com_x = xx_grid[spacing:, :] * my_grid[spacing:, :] / my_grid[spacing:, :].sum()
            com_y = yy_grid[spacing:, :] * my_grid[spacing:, :] / my_grid[spacing:, :].sum()
            
    else:
        # pattern is not at an edge crossing
        
        com_x = (xx_grid * my_grid) / my_grid.sum()
        com_y = (yy_grid * my_grid) / my_grid.sum()
    
    return com_x.sum(), com_y.sum()

def plot_grid_nbhd_update_next(grid, nbhd, update, next_grid, \
                               my_cmap=plt.get_cmap("magma"), titles=None, vmin=0.0, vmax=1):

    global subplot_0
    global subplot_1
    global subplot_2
    global subplot_3

    if titles == None:
        titles = ["CA grid time t", "Neighborhood", "Update", "CA grid time t+1"]

    fig, ax = plt.subplots(2,2, figsize=(2,2), facecolor="white")

    subplot_0 = ax[0,0].imshow(grid, cmap=my_cmap, vmin=vmin, vmax=vmax, interpolation="nearest") 
    ax[0,0].set_title(titles[0], fontsize=4)

    subplot_1 = ax[0,1].imshow(nbhd, cmap=my_cmap, vmin=vmin, vmax=vmax, interpolation="nearest")
    ax[0,1].set_title(titles[1], fontsize=4)

    my_vmin = -1.0
    my_vmax = 1.0
    subplot_2 = ax[1,0].imshow(update, cmap=my_cmap, interpolation="nearest", vmin=my_vmin, vmax=my_vmax)
    ax[1,0].set_xlabel(titles[2], fontsize=4)

    subplot_3 = ax[1,1].imshow(next_grid, cmap=my_cmap, vmin=vmin, vmax=vmax, interpolation="nearest")
    ax[1,1].set_xlabel(titles[3], fontsize=4)

    for xxyy in range(4):
        xx = xxyy // 2
        yy = xxyy % 2
        ax[xx,yy].set_xticklabels("")
        ax[xx,yy].set_yticklabels("")
    #plt.tight_layout()
    return fig, ax

def update(i):
    global grid

    n = conv_moore(grid)
    m = conv_eye(grid)

    if (use_lenia):
        # Lenia defines updates solely by a growth function without considering current state
        g = combined_growth_fn(n)
    else:
        # Generally Life-like splits growth into genesis and persistence
        g = ((1-m) * growth_fn_b(n) + m * growth_fn_s(n)) * 2 - 1
        
    subplot_0.set_array(m)
    subplot_1.set_array(n)
    subplot_2.set_array(g)
    subplot_3.set_array(grid)
    
    grid = np.clip(grid + dt * g, 0, 1.0)


In [ ]:
# life works in Lenia 

# Conway's Game of Life: Implementable in the Lenia Framework

gol_b = [3]
gol_s = [2,3]

b = my_helpers.get_smooth_interval(gol_b) 
s = my_helpers.get_smooth_interval(gol_s) 

kernel_args = {"mu": 0.99, "sigma": 0.3}
kernel_moore = my_helpers.get_kernel(my_helpers.gaussian, kernel_args, radius=1)

kernel_moore[kernel_moore > 0.01 ] = 1.0 
kernel_moore[kernel_moore <= 0.01 ] = 0.0 
kernel_moore /= kernel_moore.sum()
#kernel_moore *= 0.9

kernel_args = {"mu": 0.0, "sigma": 0.1}
kernel_eye = my_helpers.get_kernel(my_helpers.gaussian, kernel_args, radius=1)

kernel_eye[kernel_eye < 0.99 ] = 0.0 
kernel_eye /= kernel_eye.sum()

update_args = {"a": b, "alpha": 0.0125}
growth_fn_b = my_helpers.get_smooth_steps_fn(**update_args)

update_args = {"a": s, "alpha": 0.0125}
growth_fn_s = my_helpers.get_smooth_steps_fn(**update_args)

combined_growth_fn = lambda x: (growth_fn_b(x) + growth_fn_s(x)) - 1

fig = my_helpers.plot_kernel_growth(kernel_moore, combined_growth_fn, vmin=0, vmax=kernel_moore.max(), \
        titles=["Moore Kernel", "Combined Life \n Growth Function \n (Lenia)"])

# uncomment to save figure
#plt.savefig("assets/life_in_lenia.png")

plt.show()
grid = np.zeros((24,24))
#glider = my_helpers.get_puffer()

glider = np.zeros((3,3))
glider[0,1] = 1.
glider[1,1:3] = 1.
glider[2,0] = 1.
glider[2,2] = 1.

grid[10:10+glider.shape[1], 4:4+glider.shape[1]] = glider

conv_moore = my_helpers.get_conv_fn(kernel_moore)
conv_eye = my_helpers.get_conv_fn(kernel_eye)

dt = 1.0

use_lenia = True
num_frames = 64

m = grid
n = grid
g = grid 

fig, ax = plot_grid_nbhd_update_next(m, n, g, grid, my_cmap=my_cmap)
plt.close("all")

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update, frames=num_frames, interval=100).to_jshtml())

In [ ]:
# Morley: not readily mplementable in the Lenia Framework
ca = yuca.lenia.Lenia()

ca.restore_config("morley.npy")
ca.no_grad()

kernel_moore = ca.neighborhood_kernels.squeeze().numpy()

growth_fn_b = lambda x: ca.genesis_fns[0](torch.tensor(x).clone()).numpy()
growth_fn_s =  lambda x: ca.persistence_fns[0](torch.tensor(x).clone()).numpy()
combined_growth_fn = lambda x: (growth_fn_b(x) + growth_fn_s(x)) 

fig = my_helpers.plot_kernel_growth(kernel_moore, combined_growth_fn, vmin=0, vmax=kernel_moore.max(), \
        titles=["Moore Kernel", "Combined Morley \n Growth Function \n (Lenia)"])

plt.show()

grid = np.zeros((24,24))
#glider = my_helpers.get_puffer()

glider = np.zeros((6,6))
glider[2:4,0] = 1.
glider[1,1] = 1.
glider[4,1] = 1.
glider[:,2] = 1.
glider[1,3] = 1.
glider[4,3] = 1.
glider[1,4] = 1.
glider[4,4] = 1.
glider[2:4,5] = 1.

grid[10:10+glider.shape[1], 4:4+glider.shape[1]] = glider

conv_moore = my_helpers.get_conv_fn(kernel_moore)
conv_eye = my_helpers.get_conv_fn(kernel_eye)

dt = 1.0

use_lenia = True
#rter or longer animation
num_frames = 32

m = grid
n = grid
g = grid 

fig,ax = plot_grid_nbhd_update_next(m, n, g, grid, my_cmap=my_cmap)
plt.close("all")


IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update, frames=num_frames, interval=100).to_jshtml())

In [ ]:
# Morley works fine in Glaberish

ca = yuca.multiverse.CA()
ca.restore_config("morley.npy")
ca.no_grad()

kernel_moore = ca.neighborhood_kernels.squeeze().numpy()

genesis_fn = lambda x: ca.genesis_fns[0](torch.tensor(x).clone()).numpy()
persistence_fn =  lambda x: ca.persistence_fns[0](torch.tensor(x).clone()).numpy()

growth_fn_b = genesis_fn
growth_fn_s = persistence_fn

fig = my_helpers.plot_kernel_genper(kernel_moore, genesis_fn, persistence_fn, vmin=0, vmax=kernel_moore.max(), \
        titles=["Moore Kernel", "Morley \n Persistence and genesis \n (Glaberish)"])

plt.show()

grid = np.zeros((36,36))
#glider = my_helpers.get_puffer()

glider = np.zeros((6,6))
glider[2:4,0] = 1.
glider[1,1] = 1.
glider[4,1] = 1.
glider[:,2] = 1.
glider[1,3] = 1.
glider[4,3] = 1.
glider[1,4] = 1.
glider[4,4] = 1.
glider[2:4,5] = 1.

grid[10:10+glider.shape[1], 4:4+glider.shape[1]] = glider

conv_moore = my_helpers.get_conv_fn(kernel_moore)
conv_eye = my_helpers.get_conv_fn(kernel_eye)

dt = 1.0

#rter or longer animation
num_frames = 256
use_lenia = False

m = grid
n = grid
g = grid 

fig,ax = plot_grid_nbhd_update_next(m, n, g, grid, my_cmap=my_cmap)
plt.close("all")


IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update, frames=num_frames, interval=100).to_jshtml())

In [ ]:
# Figure showing examples of mobile patterns in s613 and Hydrogeminium
lib = Librarian()
lib.update_index()


torch.set_default_dtype(torch.float32)
dim_x = 192
dim_y = 192

grid_s0 = np.zeros((1, 1, dim_x, dim_y))
grid_s1 = np.zeros((1, 1, dim_x, dim_y))

grid_g0 = np.zeros((1, 1, dim_x, dim_y))
grid_g1 = np.zeros((1, 1, dim_x, dim_y))


geminium, gem_metadata = lib.load("geminium_hydrogeminium_natans000")
gem_config = gem_metadata["ca_config"]
gemwobbler, _ = lib.load("geminium2_wobble_glider000")

frog, s613_metadata = lib.load("s613_s613_frog000")
s613_config = s613_metadata["ca_config"]
wobbler, _ = lib.load("s613_fast_wobble_glider000")

ca_gem = CA()
ca_s613 = CA()

ca_gem.restore_config("geminium.npy")
ca_s613.restore_config("s613.npy")
ca_gem.no_grad()       
ca_s613.no_grad()

glider_steps = 64

grid_g0[0,0, 60:60+geminium.shape[-2], :geminium.shape[-1]] = geminium
grid_g1[0,0, 40:40+gemwobbler.shape[-2], 59:59+gemwobbler.shape[-1]] = gemwobbler
                       

grid_s0[0,0,:wobbler.shape[-2], :wobbler.shape[-1]] = wobbler
grid_s1[0,0, 80:80+frog.shape[-2], 0+wobbler.shape[-1]:0+wobbler.shape[-1]+frog.shape[-1]] = frog
                       

## update for steps

grid_g0 = torch.tensor(grid_g0).to(torch.get_default_dtype())
grid_g1 = torch.tensor(grid_g1).to(torch.get_default_dtype())

grid_s0 = torch.tensor(grid_s0).to(torch.get_default_dtype())
grid_s1 = torch.tensor(grid_s1).to(torch.get_default_dtype())

com_g0 = []
com_g1 = []
com_s0 = []
com_s1 = []

def update_gliders(ii):
    
    global grid_g0
    global grid_s0
    global grid_g1
    global grid_s1
    
    global subplot_0
    global subplot_1
    global subplot_2
    global subplot_3
    
    global com_g0
    global com_s0
    global com_g1
    global com_s1
    
    global ax
    
    grid_g0 = ca_gem(grid_g0)
    grid_g1 = ca_gem(grid_g1)
    grid_s0 = ca_s613(grid_s0)
    grid_s1 = ca_s613(grid_s1)
    
    
    if (display_center):
        
        com_g0 = com_g0[-100:]
        com_g1 = com_g1[-100:]
        com_s0 = com_s0[-100:]
        com_s1 = com_s1[-100:]
        com_g0.append(get_center_of_mass(grid_g0))   
        com_g1.append(get_center_of_mass(grid_g1))
        com_s0.append(get_center_of_mass(grid_s0))
        com_s1.append(get_center_of_mass(grid_s1))

    disp_s0 = 1.0 - my_cmap(grid_s0.cpu().numpy().squeeze())[:,:,:3]
    disp_s1 = 1.0 - my_cmap(grid_s1.cpu().numpy().squeeze())[:,:,:3]
    disp_g0 = 1.0 - my_cmap(grid_g0.cpu().numpy().squeeze())[:,:,:3]
    disp_g1 = 1.0 - my_cmap(grid_g1.cpu().numpy().squeeze())[:,:,:3]

    subplot_0.set_array(disp_g0)
    subplot_2.set_array(disp_g1)
    subplot_1.set_array(disp_s0)
    subplot_3.set_array(disp_s1)
    
    my_green = 1.0 - np.array(my_cmap(grid_g0.cpu().max().numpy()))[:3]
    my_blue = 1.0 - np.array(plt.get_cmap("afmhot")(grid_g0.cpu().max().numpy()))[:3]

    if (display_center):
        for ii in range(len(com_g0)):
            progress = ii / len(com_g0)
            my_color = progress * my_green + (1.0 - progress) * my_blue
            ax[0,0].scatter(com_g0[ii][0],com_g0[ii][1], color=my_color, s=0.5, alpha=0.05) 

        for ii in range(len(com_g1)):
            progress = ii / len(com_g1)
            my_color = progress * my_green + (1.0 - progress) * my_blue
            ax[1,0].scatter(com_g1[ii][0],com_g1[ii][1], color=my_color, s=0.5, alpha=0.05)    

        for ii in range(len(com_s0)):
            progress = ii / len(com_s0)
            my_color = progress * my_green + (1.0 - progress) * my_blue
            ax[0,1].scatter(com_s0[ii][0],com_s0[ii][1], color=my_color, s=0.5, alpha=0.05) 

        for ii in range(len(com_s1)):
            progress = ii / len(com_s1)
            my_color = progress * my_green + (1.0 - progress) * my_blue
            ax[1,1].scatter(com_s1[ii][0],com_s1[ii][1], color=my_color, s=0.5, alpha=0.05) 
        
    for xx in range(2):
        for yy in range(2):
            ax[xx, yy].set_xticklabels("")
            ax[xx, yy].set_yticklabels("")
    

disp_s0 = 1.0 - my_cmap(grid_s0.cpu().numpy().squeeze())[:,:,:3]
disp_s1 = 1.0 - my_cmap(grid_s1.cpu().numpy().squeeze())[:,:,:3]
disp_g0 = 1.0 - my_cmap(grid_g0.cpu().numpy().squeeze())[:,:,:3]
disp_g1 = 1.0 - my_cmap(grid_g1.cpu().numpy().squeeze())[:,:,:3]
    
fig, ax = plot_grid_nbhd_update_next(disp_g0, disp_s0, disp_g1, disp_s1, \
        titles=["slow_cucumber", "fast, wobbly glider", "wobbly glider", "frog"], my_cmap=my_cmap,)

plt.close("all")

display_center = False
num_frames = 512

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_gliders, frames=num_frames, interval=50).to_jshtml())

In [ ]:
# visualize spatial entropy over time

def update_spatial_entropy(ii):
    
    global grid_g
    global grid_s
    global grid_hg
    global grid_hs
    
    global subplot_0
    global subplot_1
    global subplot_2
    global subplot_3
    
    global ax
    
    grid_g = ca_gem(grid_g)
    grid_s = ca_s613(grid_s)
    
    disp_g = 1.0 - my_cmap(grid_g.cpu().numpy().squeeze())[:,:,:3]
    disp_s = 1.0 - my_cmap(grid_s.cpu().numpy().squeeze())[:,:,:3]
    
    subplot_0.set_array(disp_g)
    subplot_2.set_array(disp_s)
    
    if ii % 32 == 0:
        
        grid_hg = get_spatial_entropy(grid_g.detach().cpu().numpy().squeeze(), window_size=ca_gem.neighborhood_kernels.shape[-1])
        grid_hs = get_spatial_entropy(grid_s.detach().cpu().numpy().squeeze(), window_size=ca_s613.neighborhood_kernels.shape[-1])
        disp_hg =  my_cmap(grid_hg.squeeze()/8.)[:,:,:3]
        disp_hs =  my_cmap(grid_hs.squeeze()/8.)[:,:,:3]

        subplot_1.set_array(disp_hg)
        subplot_3.set_array(disp_hs)
        print(f"gem spatial entropy mean {np.mean(grid_hg)} +/- {np.std(grid_hg)} std. dev.")
        print(f"s613 spatial entropy mean {np.mean(grid_hs)} +/- {np.std(grid_hs)} std. dev.")
        
    for xx in range(2):
        for yy in range(2):
            ax[xx, yy].set_xticklabels("")
            ax[xx, yy].set_yticklabels("")
            
            
torch.set_default_dtype(torch.float32)

dim_x = 192
dim_y = 192
half_dim = dim_x // 4

grid_s = torch.zeros((1, 1, dim_x, dim_y))
grid_g = torch.zeros((1, 1, dim_x, dim_y))

my_init = torch.rand(1,1, 2*half_dim, 2*half_dim)

grid_s[:,:,half_dim:3 * half_dim, half_dim:3 * half_dim] = my_init 
grid_g[:,:,half_dim:3 * half_dim, half_dim:3 * half_dim] = my_init 


grid_hs = torch.zeros((1, 1, dim_x, dim_y))
grid_hg = torch.zeros((1, 1, dim_x, dim_y))

#grid_hg = get_spatial_entropy(grid_g.detach().cpu().numpy().squeeze(), window_size=ca_gem.neighborhood_kernels.shape[-1])
#grid_hs = get_spatial_entropy(grid_s.detach().cpu().numpy().squeeze(), window_size=ca_s613.neighborhood_kernels.shape[-1])
       
ca_gem = CA()
ca_s613 = CA()

ca_gem.restore_config("geminium.npy")
ca_s613.restore_config("s613.npy")
ca_gem.no_grad()       
ca_s613.no_grad()

grid_hg[0,0,0,0]= 8
grid_hs[0,0,0,0]= 8

disp_g = 1.0 - my_cmap(grid_g.squeeze())[:,:,:3]
disp_s = 1.0 - my_cmap(grid_s.squeeze())[:,:,:3]

disp_hg = my_cmap(grid_hg.squeeze())[:,:,:3]
disp_hs = my_cmap(grid_hs.squeeze())[:,:,:3]

fig, ax = plot_grid_nbhd_update_next(disp_g, disp_hg, disp_s, disp_hs, \
        titles=["${\it H. natans}$", "${\it H. natans}$ spatial entropy",\
                "s613","s613 spatial entropy"], my_cmap=my_cmap,)
plt.show()
plt.close("all")

# static figure used 512 steps, than calculate spatial entropy, 512 more steps and calculate spatial entropy again. 
num_frames = 384

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_spatial_entropy, frames=num_frames, interval=50).to_jshtml())